In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [13]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("citeseer")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.286026*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.18671026440138042


In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.12, 0.3),
        "aggregation": 'mean',
        "treatment": 'removal',#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": 10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_TESTtESTscar_val.csv",
        "min":0.72,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),
        "val":False    ,
        "methodology":"ours"     
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 21:25:30,114] A new study created in memory with name: no-name-6b2dcd10-a40b-49d4-b209-3bb0bc05f677


Running experiment with seed=654:
 - K=4, layers=2, hidden=128, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.12284701083156631, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 3.2013
Epoch 10 / 100, Loss: 2.8608
Epoch 20 / 100, Loss: 2.5826
Epoch 30 / 100, Loss: 2.4304
Epoch 40 / 100, Loss: 2.0277
Epoch 50 / 100, Loss: 1.8533
Epoch 60 / 100, Loss: 1.5734
Epoch 70 / 100, Loss: 1.4433
Epoch 80 / 100, Loss: 1.2325
Epoch 90 / 100, Loss: 1.0787


### Hyperparameter Optimization Citeseer
#### SAR 

In [4]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_test.csv",
        "min":0.745,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 21:11:08,636] A new study created in memory with name: no-name-65948645-0843-48b3-acdd-5c02de781cc9
[W 2025-04-15 21:11:08,642] Trial 0 failed with parameters: {'K': 12, 'layers': 1, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.0937795709369779, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.005074524462670685, 'anomaly_detector': 'unweighted', 'clusters': 200, 'sampling': 'shine'} because of the following error: KeyError('methodology').
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_19744\168235557.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 467, in run_nnif_gnn_experim

KeyError: 'methodology'

### Hyperparameter Optimization Cora
#### SCAR 

In [5]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("cora")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.379923*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.24134569855135768


In [3]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.38),
        "aggregation": trial.suggest_categorical("aggregation", ['mean', 'sum','max']),
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_test.csv",
        "min":0.81,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False,    
        "methodology":"ours"
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 20:50:35,153] A new study created in memory with name: no-name-e7949f3b-0b1f-4725-9108-dd65c7ee5592


Running experiment with seed=654:
 - K=24, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.34516407760669876, aggregation=sum, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 4.6933
Epoch 10 / 100, Loss: 4.4796
Epoch 20 / 100, Loss: 4.4140
Epoch 30 / 100, Loss: 4.1856
Epoch 40 / 100, Loss: 4.0219
Epoch 50 / 100, Loss: 3.9623
Epoch 60 / 100, Loss: 3.8806
Epoch 70 / 100, Loss: 3.6530
Epoch 80 / 100, Loss: 3.5389
Epoch 90 / 100, Loss: 3.3692


[I 2025-04-15 20:50:57,877] Trial 0 finished with value: 0.6592317224287485 and parameters: {'K': 24, 'layers': 2, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.34516407760669876, 'aggregation': 'sum', 'model_type': 'GATConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 0 with value: 0.6592317224287485.


 - Test Metrics: Accuracy=0.8804, F1=0.6592, Recall=0.6504, Precision=0.6683
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.81, skipping ...
Done. Results written to cora_experimentations\cora_test_1504205035.csv.
Average F1 over valid seeds: 0.6592 ± 0.0000
Running experiment with seed=654:
 - K=3, layers=1, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.30354761991701285, aggregation=sum, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 2.7790
Epoch 10 / 100, Loss: 2.0741
Epoch 20 / 100, Loss: 1.8318
Epoch 30 / 100, Loss: 1.6679
Epoch 40 / 100, Loss: 1.4837
Epoch 50 / 100, Loss: 1.3748
Epoch 60 / 100, Loss: 1.2326
Epoch 70 / 100, Loss: 1.1082
Epoch 80 / 100, Loss: 1.0117
Epoch 90 / 100, Loss: 0.9113


[I 2025-04-15 20:51:18,679] Trial 1 finished with value: 0.574468085106383 and parameters: {'K': 3, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.30354761991701285, 'aggregation': 'sum', 'model_type': 'GATConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 0 with value: 0.6592317224287485.


...

 - Test Metrics: Accuracy=0.9052, F1=0.7705, Recall=0.8949, Precision=0.6765
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.81, skipping ...
Done. Results written to cora_experimentations\cora_test_1504210916.csv.
Average F1 over valid seeds: 0.7705 ± 0.0000
Running experiment with seed=654:
 - K=13, layers=3, hidden=256, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.3012595672781328, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 2.6962
Epoch 10 / 100, Loss: 2.5939
Epoch 20 / 100, Loss: 2.4843
Epoch 30 / 100, Loss: 2.3704
Epoch 40 / 100, Loss: 2.3734
Epoch 50 / 100, Loss: 2.2339
Epoch 60 / 100, Loss: 2.2318
Epoch 70 / 100, Loss: 2.0975
Epoch 80 / 100, Loss: 2.0097
Epoch 90 / 100, Loss: 1.9373
 - Test Metrics: Accuracy=0.9400, F1=0.8376, Recall=0.8704, Precision=0.8073
 - Validation M

[I 2025-04-15 21:10:21,192] Trial 41 finished with value: 0.8133303958287434 and parameters: {'K': 13, 'layers': 3, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.3012595672781328, 'aggregation': 'mean', 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 32 with value: 0.8210146452597543.


 - Test Metrics: Accuracy=0.9265, F1=0.7890, Recall=0.7726, Precision=0.8061
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.81, skipping ...
Done. Results written to cora_experimentations\cora_test_1504210939.csv.
Average F1 over valid seeds: 0.8133 ± 0.0243
Running experiment with seed=654:
 - K=11, layers=3, hidden=256, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.2828329279033857, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 2.5652
Epoch 10 / 100, Loss: 2.4618
Epoch 20 / 100, Loss: 2.3493
Epoch 30 / 100, Loss: 2.2422
Epoch 40 / 100, Loss: 2.2331
Epoch 50 / 100, Loss: 2.1016
Epoch 60 / 100, Loss: 2.0835
Epoch 70 / 100, Loss: 1.9538
Epoch 80 / 100, Loss: 1.8595
Epoch 90 / 100, Loss: 1.7826


[W 2025-04-15 21:10:41,360] Trial 42 failed with parameters: {'K': 11, 'layers': 3, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.2828329279033857, 'aggregation': 'mean', 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_19744\112536677.py", line 40, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 568, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\t

KeyboardInterrupt: 

### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [8]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("pubmed")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.489441*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.31766718352866136


In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.31766718352866136,#trial.suggest_float("ratio", 0.2, 0.5),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048,4096]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scr_val.csv",
        "min":0.82,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False,
        "methodology":"ours"    
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 20:26:39,069] A new study created in memory with name: no-name-baf04573-d1f9-4cef-a7c5-afe48139c8cb


Running experiment with seed=654:
 - K=12, layers=2, hidden=256, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.31766718352866136, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 16.6126
Epoch 10 / 100, Loss: 14.1921
Epoch 20 / 100, Loss: 11.1615
Epoch 30 / 100, Loss: 7.9881
Epoch 40 / 100, Loss: 5.6183
Epoch 50 / 100, Loss: 4.0220
Epoch 60 / 100, Loss: 2.9893
Epoch 70 / 100, Loss: 2.3231
Epoch 80 / 100, Loss: 1.8542
Epoch 90 / 100, Loss: 1.5240


[I 2025-04-15 20:29:50,993] Trial 0 finished with value: 0.6049734009649882 and parameters: {}. Best is trial 0 with value: 0.6049734009649882.


 - Test Metrics: Accuracy=0.7976, F1=0.6050, Recall=0.6210, Precision=0.5897
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.83, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scr_val_1504202639.csv.
Average F1 over valid seeds: 0.6050 ± 0.0000
Best trial:
  Average F1: 0.6049734009649882
  Best parameters:


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [12]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("wiki-cs")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.291999*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.1822305564160316


In [15]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": 3,#trial.suggest_int("K", 2, 35),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.1822305564160316,#trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:11:10,847] A new study created in memory with name: no-name-1f95e279-aada-49a9-a5d2-2cdb69773da7


Running experiment with seed=654:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.6099
Epoch 10 / 100, Loss: 3.8026
Epoch 20 / 100, Loss: 3.1336
Epoch 30 / 100, Loss: 2.5338
Epoch 40 / 100, Loss: 2.0711
Epoch 50 / 100, Loss: 1.7149
Epoch 60 / 100, Loss: 1.4327
Epoch 70 / 100, Loss: 1.2042
Epoch 80 / 100, Loss: 1.0330
Epoch 90 / 100, Loss: 0.8866
 - Test Metrics: Accuracy=0.9615, F1=0.9182, Recall=0.9425, Precision=0.8951
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 -

[I 2025-04-14 22:20:59,610] Trial 0 finished with value: 0.9146052245003616 and parameters: {}. Best is trial 0 with value: 0.9146052245003616.


 - Test Metrics: Accuracy=0.9592, F1=0.9127, Recall=0.9302, Precision=0.8958
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1404221110.csv.
Average F1 over valid seeds: 0.9146 ± 0.0020
Best trial:
  Average F1: 0.9146052245003616
  Best parameters:


### Hyperparameter Optimization WikiCS
#### SAR 

In [14]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 3,#trial.suggest_int("K", 2, 35),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.1822305564160316,#trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:00:29,047] A new study created in memory with name: no-name-1a4f9b7d-322b-4c27-bab1-ba7a9f3fd599


Running experiment with seed=654:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.5456
Epoch 10 / 100, Loss: 3.7924
Epoch 20 / 100, Loss: 3.0968
Epoch 30 / 100, Loss: 2.5100
Epoch 40 / 100, Loss: 2.0480
Epoch 50 / 100, Loss: 1.6982
Epoch 60 / 100, Loss: 1.4190
Epoch 70 / 100, Loss: 1.1968
Epoch 80 / 100, Loss: 1.0250
Epoch 90 / 100, Loss: 0.8854
 - Test Metrics: Accuracy=0.9599, F1=0.9145, Recall=0.9362, Precision=0.8938
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 -

[I 2025-04-14 22:11:10,444] Trial 0 finished with value: 0.9125326523955319 and parameters: {}. Best is trial 0 with value: 0.9125326523955319.


 - Test Metrics: Accuracy=0.9564, F1=0.9070, Recall=0.9287, Precision=0.8864
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to wiki-cs_experimentations\wikics_sar_val_1404220029.csv.
Average F1 over valid seeds: 0.9125 ± 0.0028
Best trial:
  Average F1: 0.9125326523955319
  Best parameters:


# Elliptic Bitcoin

In [ ]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("elliptic-bitcoin")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
#0.018961
ratio=(0.02*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.008049935058661887


In [ ]:
# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
#0.018961
ratio=(0.022062*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

#66.1

0.010137078970813966


In [12]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": 6,#trial.suggest_int("K", 2, 10),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 64,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,#trial.suggest_float("dropout", 0, 0.4),
        "ratio": 0.010137078970813966,#trial.suggest_float("ratio", 0.01, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": 'GCNConv',#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [4096,8192,16384]),
        "lr": 0.005,#trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.6,
        "clusters":50,#trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":50,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"sage",#trial.suggest_categorical("sampling",["neighbor","cluster"]),   
        "val":False      
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 17:54:20,375] A new study created in memory with name: no-name-3fbbd5a8-5d67-47c1-9323-62e661673c34


Running experiment with seed=654:
 - K=6, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.011150786867895364, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 50, Loss: 132.0312
Epoch 10 / 50, Loss: 12.5536
Epoch 20 / 50, Loss: 3.7619
Epoch 30 / 50, Loss: 1.6689
Epoch 40 / 50, Loss: 0.8631


[I 2025-04-15 18:13:15,186] Trial 0 finished with value: 0.6594922737306843 and parameters: {}. Best is trial 0 with value: 0.6594922737306843.


 - Test Metrics: Accuracy=0.9470, F1=0.6595, Recall=0.5259, Precision=0.8842
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1504175420.csv.
Average F1 over valid seeds: 0.6595 ± 0.0000
Running experiment with seed=654:
 - K=6, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.011150786867895364, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005


[W 2025-04-15 18:14:11,889] Trial 1 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_21344\1434882542.py", line 42, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 562, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 297, in train_graph
    neg_mask, pos_mask = nnif_detector.get_reliable(features_np, y_labels)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
names=['cora','citeseer','pubmed','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['naive','two_nnif','spy','nnpu','imbnnpu']:
            for model_type in ['MLP','SAGEConv']:
                if methodology in ['naive','two_nnif','spy']:
                    out_channels=2
                else:
                    out_channels=1
                print(methodology)
                params = {
                    "dataset_name": name,
                    "mechanism": mechanism,
                    "train_pct": 0.5,
                    "K": 4,
                    "layers": 2,
                    "hidden_channels": 16,
                    "out_channels": out_channels,
                    "norm": None,
                    "dropout": 0,
                    "ratio": ratios[i],
                    "aggregation": 'mean',
                    "treatment": "removal",
                    "model_type": model_type,
                    "rate_pairs":10,
                    "batch_size": 2048,
                    "lr": 0.005,
                    "anomaly_detector":"nearest_neighbors",
                    "seeds": 5,
                    "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                    "min":0.0,
                    "clusters":50,        
                    "num_epochs":50,         
                    "sampling":"sage",
                    "val":False,
                    "methodology": methodology,
                }
                avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                print('___________________________________________________________________________________')
                print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
                print('___________________________________________________________________________________')

In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
names=['cora','citeseer','pubmed','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['XGBoost','naive', 'NNIF','two_nnif','spy', 'nnpu','imbnnpu']:
            if methodology in ['naive','two_nnif','spy']:
                out_channels=2
            else:
                out_channels=1
            print(methodology)
            params = {
                "dataset_name": name,
                "mechanism": mechanism,
                "train_pct": 0.5,
                "K": 4,
                "layers": 2,
                "hidden_channels": 16,
                "out_channels": out_channels,
                "norm": None,
                "dropout": 0,
                "ratio": ratios[i],
                "aggregation": 'mean',
                "treatment": "removal",
                "model_type": 'GCNConv',
                "rate_pairs":10,
                "batch_size": 2048,
                "lr": 0.005,
                "anomaly_detector":"nearest_neighbors",
                "seeds": 5,
                "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                "min":0.0,
                "clusters":50,        
                "num_epochs":50,         
                "sampling":"sage",
                "val":False,
                "methodology": methodology,
            }
            avg_f1, std_f1 = run_nnif_gnn_experiment(params)
            print('___________________________________________________________________________________')
            print(f"Avg F1 for {name} with {mechanism} and {methodology}: {avg_f1:.4f} ± {std_f1:.4f}")
            print('___________________________________________________________________________________')

XGBoost
Running experiment with seed=654:
 - K=4, layers=2, hidden=16, out=1
 - norm=None, dropout=0, batch_size=2048, methodology=XGBoost
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here 2
 - Test Metrics: Accuracy=0.8282, F1=0.0662, Recall=0.0342, Precision=1.0000
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=4, layers=2, hidden=16, out=1
 - norm=None, dropout=0, batch_size=2048, methodology=XGBoost
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here 2
 - Test Metrics: Accuracy=0.8256, F1=0.0384, Recall=0.0196, Precision=1.0000
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=25:
 - K=4, layers=

KeyboardInterrupt: 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 7),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_sampling_nnif_batch_cluster.csv",
        "min":0.845,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
